In [ ]:
import gffutils
import os
from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool
from pprint import pprint as pp
import time
import itertools
from src.utility_functions import *

In [ ]:
gff_path = "input/gff/"
fasta_path = "input/fasta"
db_path = "intermediate/"
species_list = ["Bcur", "Bdor", "Bole", "Ccap"]
ortho_group_path = "input/new_groups_filter.txt"

In [ ]:
start = time.clock()
print("created?\n" +
      "--------")
with Pool(len(species_list)) as p: results = {sp:db for sp,db in p.starmap(create_db, zip(itertools.repeat(db_path), itertools.repeat(gff_path), species_list))}
for sp,status in results.items():
    print("{}: {}".format(sp, status))
end = time.clock()
print("time: {}".format(end - start))

In [ ]:
start = time.clock()
gff_dbs = {sp:db for sp,db in [connect_db(db_path, sp) for sp in species_list]}
exons = {}
for sp in species_list:
    exons[sp] = gff_dbs[sp].count_features_of_type('exon')
    print("{} exons: {}".format(sp, exons[sp]))
end = time.clock()
print("time: {}".format(end - start))

In [ ]:
get_ortho_groups(ortho_group_path)